## Inference code for YellowKing's model from  DL Sprint 2022
https://www.kaggle.com/code/sameen53/yellowking-dlsprint-inference

In [ ]:
!cp -r ../input/python-packages2 ./

In [ ]:
!tar xvfz ./python-packages2/jiwer.tgz
!pip install ./jiwer/jiwer-2.3.0-py3-none-any.whl -f ./ --no-index
!tar xvfz ./python-packages2/normalizer.tgz
!pip install ./normalizer/bnunicodenormalizer-0.0.24.tar.gz -f ./ --no-index
!tar xvfz ./python-packages2/pyctcdecode.tgz
!pip install ./pyctcdecode/attrs-22.1.0-py2.py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/exceptiongroup-1.0.0rc9-py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/hypothesis-6.54.4-py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/pygtrie-2.5.0.tar.gz -f ./ --no-index --no-deps
!pip install ./pyctcdecode/sortedcontainers-2.4.0-py2.py3-none-any.whl -f ./ --no-index --no-deps
!pip install ./pyctcdecode/pyctcdecode-0.4.0-py2.py3-none-any.whl -f ./ --no-index --no-deps

!tar xvfz ./python-packages2/pypikenlm.tgz
!pip install ./pypikenlm/pypi-kenlm-0.1.20220713.tar.gz -f ./ --no-index --no-deps



jiwer/
jiwer/jiwer-2.3.0-py3-none-any.whl
jiwer/python-Levenshtein-0.12.2.tar.gz
jiwer/setuptools-65.3.0-py3-none-any.whl
Looking in links: ./
Processing ./jiwer/jiwer-2.3.0-py3-none-any.whl
normalizer/
normalizer/bnunicodenormalizer-0.0.24.tar.gz
Looking in links: ./
Processing ./normalizer/bnunicodenormalizer-0.0.24.tar.gz
  Preparing metadata (setup.py) ... done
  Created wheel for bnunicodenormalizer: filename=bnunicodenormalizer-0.0.24-py3-none-any.whl size=17628 sha256=36d157b3150eaf9ef2396c4aaebaad92817ed13e848bcafbd9e98b11ac2a6be1
  Stored in directory: /root/.cache/pip/wheels/3f/7a/fd/fde270138c2a0d8efd0e9ce20f760c2e3a0fc4317975237cb8
Successfully built bnunicodenormalizer
pyctcdecode/
pyctcdecode/hypothesis-6.54.4-py3-none-any.whl
pyctcdecode/sortedcontainers-2.4.0-py2.py3-none-any.whl
pyctcdecode/exceptiongroup-1.0.0rc9-py3-none-any.whl
pyctcdecode/pyctcdecode-0.4.0-py2.py3-none-any.whl
pyctcdecode/numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
pyctcd

In [ ]:
!pip install openpyxl

ERROR: Could not find a version that satisfies the requirement openpyxl (from versions: none)
ERROR: No matching distribution found for openpyxl


In [ ]:
import os
import numpy as np
from tqdm.auto import tqdm
from glob import glob
from transformers import AutoFeatureExtractor, pipeline
import pandas as pd
import librosa
import IPython
from datasets import load_metric
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader
import torch
import gc
import wave
from scipy.io import wavfile
import scipy.signal as sps
import pyctcdecode


tqdm.pandas()
import warnings
warnings.filterwarnings("ignore")



In [ ]:
# CHANGE ACCORDINGLY
BATCH_SIZE = 16
TEST_DIRECTORY1 = '/kaggle/input/kishoreganj-1'

path1 = glob(os.path.join(TEST_DIRECTORY1,'*.wav'))
paths = path1
#print(paths[:2])

In [ ]:
class CFG:
    my_model_name = '../input/yellowking-dlsprint-model/YellowKing_model'
    processor_name = '../input/yellowking-dlsprint-model/YellowKing_processor'


from transformers import Wav2Vec2ProcessorWithLM

processor = Wav2Vec2ProcessorWithLM.from_pretrained(CFG.processor_name)

In [ ]:
my_asrLM = pipeline("automatic-speech-recognition", model=CFG.my_model_name ,feature_extractor =processor.feature_extractor, tokenizer= processor.tokenizer,decoder=processor.decoder ,device=0)


speech, sr = librosa.load('/kaggle/input/bengaliai-speech/test_mp3s/0f3dac00655e.mp3', sr=processor.feature_extractor.sampling_rate)

my_asrLM([speech]*2, chunk_length_s=112, stride_length_s=None)

my_asrLM



class AudioDataset(Dataset):
    def __init__(self, paths):
        self.paths = paths
    def __len__(self):
        return len(self.paths)
    def __getitem__(self,idx):
        speech, sr = librosa.load(self.paths[idx], sr=processor.feature_extractor.sampling_rate)
#         print(speech.shape)
        return speech


dataset = AudioDataset(paths)
dataset[0]
device = 'cuda:0'

def collate_fn_padd(batch):
    '''
    Padds batch of variable length

    note: it converts things ToTensor manually here since the ToTensor transform
    assume it takes in images rather than arbitrary tensors.
    '''
    ## get sequence lengths
    lengths = torch.tensor([ t.shape[0] for t in batch ])
    ## padd
    batch = [ torch.Tensor(t) for t in batch ]
    batch = torch.nn.utils.rnn.pad_sequence(batch)
    ## compute mask
    mask = (batch != 0)
    return batch, lengths, mask


dataloader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=8, collate_fn=collate_fn_padd)



preds_all = []
for batch, lengths, mask in dataloader:
    preds = my_asrLM(list(batch.numpy().transpose()))
    preds_all+=preds


    from bnunicodenormalizer import Normalizer


bnorm = Normalizer()
def normalize(sen):
    _words = [bnorm(word)['normalized']  for word in sen.split()]
    return " ".join([word for word in _words if word is not None])

def dari(sentence):
    try:
        if sentence[-1]!="।":
            sentence+="।"
    except:
        print(sentence)
    return sentence




df1= pd.DataFrame(
    {
        "External_ID":[p.split(os.sep)[-1].replace('.wav','') for p in paths],
        "content":[p['text']for p in preds_all]
    }
)
df1.content= df1.content.apply(lambda x:normalize(x))
df1.content= df1.content.apply(lambda x:dari(x))

In [ ]:
df1

,External_ID,content
0,rec_04_audio_44,কারবারিরা কে গেলে কাকে সাদি।
1,rec_19_audio_24,সানিকে অবতরনকরবার দেশ।
2,rec_05_audio_0,মলিকুলার জমবে কিশোরগঞ্জে যা সবচেয়ে বড় মশিদপুণ্...
3,rec_05_audio_18,প্রতিটি জমিদাতা মাহাসিন মাসের দারকে পরে এই বরা...
4,rec_18_audio_10,তাতে দিনের নেপালদেবকে না।
...,...,...
995,rec_23_audio_35,কমান্ডার কিনারা।
996,rec_09_audio_19,মাগুরা লাগুর ব্যবসা কাগজখানি নারিলা গর্বের কের...
997,rec_04_audio_6,কাশিকার দেবোত্তর।
998,rec_16_audio_7,তমিজের করিনা।


In [ ]:
df1.to_csv("submission1.csv", index=False)


In [ ]:
# CHANGE ACCORDINGLY
BATCH_SIZE = 16
TEST_DIRECTORY2 = '/kaggle/input/kishorehanj-2'
path2 = glob(os.path.join(TEST_DIRECTORY2,'*.wav'))
paths = path2

In [ ]:
my_asrLM = pipeline("automatic-speech-recognition", model=CFG.my_model_name ,feature_extractor =processor.feature_extractor, tokenizer= processor.tokenizer,decoder=processor.decoder ,device=0)


#speech, sr = librosa.load('/kaggle/input/bengaliai-speech/test_mp3s/0f3dac00655e.mp3', sr=processor.feature_extractor.sampling_rate)

my_asrLM([speech]*2, chunk_length_s=112, stride_length_s=None)

my_asrLM



class AudioDataset(Dataset):
    def __init__(self, paths):
        self.paths = paths
    def __len__(self):
        return len(self.paths)
    def __getitem__(self,idx):
        speech, sr = librosa.load(self.paths[idx], sr=processor.feature_extractor.sampling_rate)
#         print(speech.shape)
        return speech


dataset = AudioDataset(paths)
#dataset[0]
device = 'cuda:0'

def collate_fn_padd(batch):
    '''
    Padds batch of variable length

    note: it converts things ToTensor manually here since the ToTensor transform
    assume it takes in images rather than arbitrary tensors.
    '''
    ## get sequence lengths
    lengths = torch.tensor([ t.shape[0] for t in batch ])
    ## padd
    batch = [ torch.Tensor(t) for t in batch ]
    batch = torch.nn.utils.rnn.pad_sequence(batch)
    ## compute mask
    mask = (batch != 0)
    return batch, lengths, mask


dataloader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=8, collate_fn=collate_fn_padd)



preds_all = []
for batch, lengths, mask in dataloader:
    preds = my_asrLM(list(batch.numpy().transpose()))
    preds_all+=preds


    from bnunicodenormalizer import Normalizer


bnorm = Normalizer()
def normalize(sen):
    _words = [bnorm(word)['normalized']  for word in sen.split()]
    return " ".join([word for word in _words if word is not None])

def dari(sentence):
    try:
        if sentence[-1]!="।":
            sentence+="।"
    except:
        print(sentence)
    return sentence




df2= pd.DataFrame(
    {
        "External_ID":[p.split(os.sep)[-1].replace('.wav','') for p in paths],
        "content":[p['text']for p in preds_all]
    }
)

df2.content= df2.content.apply(lambda x:normalize(x))
df2.content= df2.content.apply(lambda x:dari(x))

In [ ]:
df2

,External_ID,content
0,rec_32_audio_29,এ তাকে আগে ভর্তকি আমি গেছি রামপুর গেছে কি দীর্...
1,rec_43_audio_21,বরফ মিলটিকে কারকারেকে যা তুমি তাদের তা গণদেবিভ...
2,rec_26_audio_43,সুনাম কিছু নাহি তার কিছু উদ্দি হাসি কোন কিটিকে...
3,rec_42_audio_4,গরু বা সুচিন্তা নিয়া গৌরব মা চা কানে আওয়ার জাত...
4,rec_35_audio_36,চমকাবেন বাহিনী না বাজালে পরিবার বা ভিসা পরিসেব...
...,...,...
995,rec_43_small_audio_0,কলা রেমাক্রি।
996,rec_29_audio_10,সারা নাগেরবাজারের।
997,rec_45_audio_11,কান্দি কনিনিকা।
998,rec_26_audio_51,চল চল চল কাজীনজরুল ইসলাম চল চল চল অদ্য গগনে বা...


In [ ]:
df2.to_csv("submission2.csv", index=False)


In [ ]:
# CHANGE ACCORDINGLY
BATCH_SIZE = 16
TEST_DIRECTORY3 = '/kaggle/input/kishoreganj-3'
path3 = glob(os.path.join(TEST_DIRECTORY3,'*.wav'))
paths = path3

In [ ]:
my_asrLM = pipeline("automatic-speech-recognition", model=CFG.my_model_name ,feature_extractor =processor.feature_extractor, tokenizer= processor.tokenizer,decoder=processor.decoder ,device=0)


#speech, sr = librosa.load('/kaggle/input/bengaliai-speech/test_mp3s/0f3dac00655e.mp3', sr=processor.feature_extractor.sampling_rate)

my_asrLM([speech]*2, chunk_length_s=112, stride_length_s=None)

my_asrLM



class AudioDataset(Dataset):
    def __init__(self, paths):
        self.paths = paths
    def __len__(self):
        return len(self.paths)
    def __getitem__(self,idx):
        speech, sr = librosa.load(self.paths[idx], sr=processor.feature_extractor.sampling_rate)
#         print(speech.shape)
        return speech


dataset = AudioDataset(paths)
#dataset[0]
device = 'cuda:0'

def collate_fn_padd(batch):
    '''
    Padds batch of variable length

    note: it converts things ToTensor manually here since the ToTensor transform
    assume it takes in images rather than arbitrary tensors.
    '''
    ## get sequence lengths
    lengths = torch.tensor([ t.shape[0] for t in batch ])
    ## padd
    batch = [ torch.Tensor(t) for t in batch ]
    batch = torch.nn.utils.rnn.pad_sequence(batch)
    ## compute mask
    mask = (batch != 0)
    return batch, lengths, mask


dataloader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=8, collate_fn=collate_fn_padd)



preds_all = []
for batch, lengths, mask in dataloader:
    preds = my_asrLM(list(batch.numpy().transpose()))
    preds_all+=preds


    from bnunicodenormalizer import Normalizer


bnorm = Normalizer()
def normalize(sen):
    _words = [bnorm(word)['normalized']  for word in sen.split()]
    return " ".join([word for word in _words if word is not None])

def dari(sentence):
    try:
        if sentence[-1]!="।":
            sentence+="।"
    except:
        print(sentence)
    return sentence


df3= pd.DataFrame(
    {
        "External_ID":[p.split(os.sep)[-1].replace('.wav','') for p in paths],
        "content":[p['text']for p in preds_all]
    }
)



df3.content= df3.content.apply(lambda x:normalize(x))
df3.content= df3.content.apply(lambda x:dari(x))


In [ ]:
df3

,External_ID,content
0,rec_47_audio_67,সায়েরাকে করা বা বিনা মিরকামারি মুক্তি অভাবে।
1,rec_47_audio_49,কাতানিয়া হয়েছে কালকা মেল আগে পালা।
2,rec_46_audio_36,মাকে বিমান আছে বিমান বেলবানা তথ্য বাড়িবাকার বি...
3,rec_47_audio_28,কে তাকে লালবাজারের মানুষকে বালাকোটে বা।
4,rec_47_audio_34,মিকাকে।
...,...,...
80,rec_47_audio_7,দারপ্রান্তে মেছতায় কবাকের পাকতে আছে।
81,rec_47_audio_30,একজন বিচারক না লেবাসে ব্যবহারকে মেমনের বাহিনী ...
82,rec_46_audio_38,কে।
83,rec_47_audio_0,ইয়ালেন ওয়েস্টল্যান্ড বা পেতিতে গ্যাসকাটার কাজে...


In [ ]:
df3.to_csv("submission3.csv", index=False)


In [ ]:
print('done')

done


In [ ]:
# import pandas as pd
# path1 = "submission1.csv"
# path2 = "submission2.csv"
# path3 = "submission3.csv"


# df1 = pd.read_csv(path1)
# df2 = pd.read_csv(path2)
# df3 = pd.read_csv(path3)

values1 = df1[['External_ID','content']]
values2 = df2[['External_ID','content']]
values3 = df3[['External_ID','content']]


dataframes = [values1, values2, values3]

join = pd.concat(dataframes)
join

,External_ID,content
0,rec_04_audio_44,কারবারিরা কে গেলে কাকে সাদি।
1,rec_19_audio_24,সানিকে অবতরনকরবার দেশ।
2,rec_05_audio_0,মলিকুলার জমবে কিশোরগঞ্জে যা সবচেয়ে বড় মশিদপুণ্...
3,rec_05_audio_18,প্রতিটি জমিদাতা মাহাসিন মাসের দারকে পরে এই বরা...
4,rec_18_audio_10,তাতে দিনের নেপালদেবকে না।
...,...,...
80,rec_47_audio_7,দারপ্রান্তে মেছতায় কবাকের পাকতে আছে।
81,rec_47_audio_30,একজন বিচারক না লেবাসে ব্যবহারকে মেমনের বাহিনী ...
82,rec_46_audio_38,কে।
83,rec_47_audio_0,ইয়ালেন ওয়েস্টল্যান্ড বা পেতিতে গ্যাসকাটার কাজে...


In [ ]:
join.to_csv("2.2: kishoreganj-yellowking-inference.csv")

In [ ]:
print('done')

done
